In [ ]:
from google.colab import drive
import pandas as pd
import re

drive.mount('/content/drive')

In [ ]:
def remove_emojis(text):
    """
    Remove emojis de um texto.

    Args:
        text (str): O texto a ser processado.

    Returns:
        str: O texto sem emojis.
    """

    if isinstance(text, str):
        emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"
            "\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF"
            "\U0001F1E0-\U0001F1FF"
            "\U00002500-\U00002BEF"
            "\U00002702-\U000027B0"
            "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251"
            "\U0001f926-\U0001f937"
            "\U00010000-\U0010ffff"
            "\u200d"
            "\u2640-\u2642"
            "\u2600-\u2B55"
            "\u23cf"
            "\u23e9"
            "\u231a"
            "\u3030"
            "\ufe0f"
            "]+", flags=re.UNICODE)

        return emoji_pattern.sub(r'', text)
    else:
        return text

def standardize_columns(df):
  """
  Padroniza as colunas de um DataFrame.

  Args:
      df (pd.DataFrame): O DataFrame a ser processado.

  Returns:
      pd.DataFrame: O DataFrame com as colunas padronizadas.
  """
  if 'Resposta\n' in df.columns:
    df = df.rename(columns={'Resposta\n': 'Resposta'})
  if 'Unnamed: 0' in df.columns:
    df = df.rename(columns={'Unnamed: 0': 'No'})
  if 'Comentario do Inacio' in df.columns:
    df = df.drop(columns=['Comentario do Inacio'])

  df = df.dropna(subset=['Pergunta', 'Resposta'], how='all')
  df = df.replace('\n', ' ', regex=True)

  df['Pergunta'] = df['Pergunta'].apply(remove_emojis)
  df['Resposta'] = df['Resposta'].apply(remove_emojis)

  return df

In [ ]:
conv1 = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Conversas1.csv"
conv2 = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Conversas2.csv"
qa = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Q&A2.csv"
dictionary = "/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/Dicionario.csv"
old_df = "/content/drive/Shareddrives/grupo3moshi/chats_clients.csv"


# Padroniza as colunas para depois conseguir concatenar todos os dataframes
conv1_df = standardize_columns(pd.read_csv(conv1))
conv2_df = standardize_columns(pd.read_csv(conv2))
qa_df = standardize_columns(pd.read_csv(qa, on_bad_lines='skip'))
dictionary_df = pd.read_csv(dictionary)
old_df = standardize_columns(pd.read_csv(old_df))

In [ ]:
# Continua o número do chat a partir do último do dataframe antigo
qa_df['No'] = range(len(old_df) + 1, len(old_df) + 1 + len(qa_df))

# Concatena o dataframe de QA e o da sprint 2
combined_qa_df = pd.concat([old_df, qa_df], ignore_index=True)
print(len(combined_qa_df))

In [ ]:
def combine_questions_and_answers(df, no_start):
  """
  Combina as respostas e perguntas em uma única coluna, de uma forma que tenha
  o histórico do chat até aquele momento.

  Args:
      df (pd.DataFrame): O DataFrame a ser processado.
      no_start (int): O número inicial para o No (número do chat).

  Returns:
      pd.DataFrame: O DataFrame com as perguntas e respostas combinadas.
  """
  df['No'] = df['No'].fillna(method='ffill')
  df['Intencao'] = df['Intencao'].fillna(method='ffill')

  unique_numbers = df['No'].unique()

  # Para cada chat, junta as perguntas e respostas, criando um histórico da
  # conversa até aquela pergunta
  for number in unique_numbers:
    full_string = ""
    first_line = True
    rows = df.loc[df['No'] == number, ['Pergunta', 'Resposta']]
    for index, row in rows.iterrows():
      if first_line:
        full_string += f"{row['Pergunta']} {row['Resposta']} "
        first_line = False
        continue
      if pd.notna(row['Pergunta']):
        full_string += f"{row['Pergunta']} "
      df.loc[index, 'Pergunta'] = full_string
      if pd.notna(row['Resposta']):
        full_string += f"{row['Resposta']} "


  mapping = {old_value: i + no_start for i, old_value in enumerate(unique_numbers)}
  df['No'] = df['No'].map(mapping)

  return df

conv2_df = combine_questions_and_answers(conv2_df, len(combined_qa_df) + 1)
conv1_df = combine_questions_and_answers(conv1_df, len(conv2_df) + len(combined_qa_df) + 1)

In [ ]:
combined_df = pd.concat([combined_qa_df, conv2_df], ignore_index=True)
combined_df = pd.concat([combined_df, conv1_df], ignore_index=True)

In [ ]:
substituicoes = dict(zip(dictionary_df['Palavra '], dictionary_df['Significado']))

def substituir_palavras(texto, substituicoes):
    """
    Substitui palavras em um texto com base no dicionário mandado pelo Inácio
    """
    for palavra, significado in substituicoes.items():
        texto = texto.replace(palavra, significado)
    return texto

combined_df['Pergunta'] = combined_df['Pergunta'].apply(lambda x: substituir_palavras(x, substituicoes))

In [ ]:
combined_df

In [ ]:
combined_df.to_csv('/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/data_sprint_3.csv', index=False)